In [2]:
import pandas as pd
import vaex
import numpy as np
import matplotlib.pyplot as plt

/home/joshua/.local/lib/python3.8/site-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 0.1.36ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(
/home/joshua/.local/lib/python3.8/site-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 0.23ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(


In [3]:
import re
from sklearn.pipeline import Pipeline
import spacy
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
word_lemmatizer = WordNetLemmatizer()
nlp = spacy.load("en_core_web_sm")

In [4]:
from sklearn.base import TransformerMixin,BaseEstimator

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV,StratifiedShuffleSplit,RandomizedSearchCV,StratifiedKFold
from sklearn.svm import LinearSVC
from sklearn.svm import SVC

In [7]:
from sklearn.metrics import mean_squared_error

In [8]:
sentiment140_df = pd.read_csv('training140.csv',encoding='latin-1',header=None)

In [9]:
from vaex.dataset import encoding


#s140_df = vaex.open('training140.csv',encoding='latin-1')
s140_df = vaex.from_csv('training140.csv',names=['col1','col2','col3','col4','col5','col6'],encoding = 'latin-1')

In [10]:
sentiment140_df.head()

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [11]:
s140_df.head()

#,col1,col2,col3,col4,col5,col6
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"'@switchfoot http://twitpic.com/2y1zl - Awww, th..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,"""is upset that he can't update his Facebook by t..."
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,'@Kenichan I dived many times for the ball. Mana...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"""@nationwideclass no, it's not behaving at all. ..."
5,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew
6,0,1467811592,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,mybirch,Need a hug
7,0,1467811594,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,coZZ,'@LOLTrish hey long time no see! Yes.. Rains a ...
8,0,1467811795,Mon Apr 06 22:20:05 PDT 2009,NO_QUERY,2Hood4Hollywood,@Tatiana_K nope they didn't have it
9,0,1467812025,Mon Apr 06 22:20:09 PDT 2009,NO_QUERY,mimismo,@twittera que me muera ?


In [12]:
s140_df.shape

(1600000, 6)

In [13]:
s140_df

#,col1,col2,col3,col4,col5,col6
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"'@switchfoot http://twitpic.com/2y1zl - Awww, th..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,"""is upset that he can't update his Facebook by t..."
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,'@Kenichan I dived many times for the ball. Mana...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"""@nationwideclass no, it's not behaving at all. ..."
...,...,...,...,...,...,...
"1,599,995",4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,'Just woke up. Having no school is the best feel...
"1,599,996",4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,'TheWDB.com - Very cool to hear old Walt intervi...
"1,599,997",4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,'Are you ready for your MoJo Makeover? Ask me fo...
"1,599,998",4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,'Happy 38th Birthday to my boo of alll time!!! T...


In [18]:
s140_df[['col6','col1']]

#,col6,col1
0,"'@switchfoot http://twitpic.com/2y1zl - Awww, th...",0
1,"""is upset that he can't update his Facebook by t...",0
2,'@Kenichan I dived many times for the ball. Mana...,0
3,my whole body feels itchy and like its on fire,0
4,"""@nationwideclass no, it's not behaving at all. ...",0
...,...,...
"1,599,995",'Just woke up. Having no school is the best feel...,4
"1,599,996",'TheWDB.com - Very cool to hear old Walt intervi...,4
"1,599,997",'Are you ready for your MoJo Makeover? Ask me fo...,4
"1,599,998",'Happy 38th Birthday to my boo of alll time!!! T...,4


In [21]:
df1=s140_df['col1']
df1.columns = ['sentiment','analysis']
df1.dropna(axis=0,how='any')

TypeError: dropna() got an unexpected keyword argument 'axis'

In [54]:
sentiment140_df.shape

(1600000, 6)

In [55]:
print(sentiment140_df[0].value_counts())

0    800000
4    800000
Name: 0, dtype: int64


In [56]:
df = sentiment140_df[[5,0]]
df.columns = ['sentiment','analysis']
df = df.dropna(axis=0,how='any')

In [57]:
X_train = df['sentiment'].to_frame()
y_train = df['analysis'].to_frame()

In [58]:
def preprocess(sentence):
    sentence = re.sub('<[^>]*>','',sentence)
    emoticons = re.findall(':-?[()]',sentence)
    sentence = re.sub('[\W]+|[\d]+',' ',sentence.lower()) + ' '.join(emoticons)
    return sentence

def lemmatizer(sentence):
    '''
    lem_sent = []
    for word in nlp(sentence):
        lem_sent.append(word.lemma_)
    return ''.join(lem_sent)
    '''
    return ''.join([word_lemmatizer.lemmatize(word) for word in sentence])

def stop_words_remover(sentence):
    sentence = ''.join(sentence)
    stopwords = nlp.Defaults.stop_words
    new_sent = ''
    for word_token in sentence.split():
        if word_token not in stopwords:
            new_sent = new_sent + word_token + ' '
    return new_sent
    '''
    stop_words = stopwords.words('english')
    return ''.join([word for word in sentence if word not in stop_words])
    '''

In [59]:
class DataCleaner(BaseEstimator,TransformerMixin):
    def __init__(self,X,y=None):
        self.X = X
        self.y = y
        
    
    def fit(self,X,y=None):
        return self

    def transform(self,X,y=None):
        X_ = X.copy()
        for i in range(X_.shape[0]):
            X_.iloc[i] = preprocess(X_.iloc[i])
            X_.iloc[i] = stop_words_remover(X_.iloc[i])
            X_.iloc[i] = lemmatizer(X_.iloc[i])
        return X_

In [60]:
tfidf = TfidfVectorizer(lowercase=False,
                        #preprocessor=preprocess,
                        #tokenizer=lemmatizer,
                        #stop_words=nlp.Defaults.stop_words,
                        use_idf=True,
                        norm='l2',
                        smooth_idf=True
                        )

In [61]:
clf1 = LinearSVC()
clf2 = SVC()

In [62]:
param_grid1 =   {'classifier__C':[0.01,0.1,1.0,10.0,100.0],
                
                }

param_grid2 =   {'classifier__C':[0.01,0.1,1.0,10.0,100.0],
                'classifier__gamma':[0.01,0.1,1.0,10.0,100.0],
                'classifier__kernel':['rbf','sigmoid']
                }

In [63]:
cv = StratifiedKFold(n_splits=10,shuffle=True,random_state=39)

In [64]:
pipe = Pipeline([
                ('data_cleaning',DataCleaner(X_train)),
                ('vectorizer',tfidf),
                ('classifier',clf1)
        ])

In [65]:
random_search_svc = RandomizedSearchCV(estimator = pipe, 
                                        param_distributions=param_grid1,
                                        cv=10,
                                        refit= 'acc',
                                        scoring={'acc':'accuracy',
                                                 'mse':'neg_mean_squared_error'
                                                },
                                        n_jobs=6,
                                        return_train_score=True 
                                )

In [66]:
y_train['analysis'].isnull().value_counts()

False    1600000
Name: analysis, dtype: int64

In [67]:
random_search_svc.fit(X_train,y_train)

/home/joshua/.local/lib/python3.8/site-packages/sklearn/model_selection/_search.py:306: UserWarning: The total space of parameters 5 is smaller than n_iter=10. Running 5 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


: 

: 